In [1]:
import numpy as np
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
import yaml
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
with open("models_hyperparameters.yaml", "r") as stream:
    models_hyperparameters = yaml.safe_load(stream)
    
with open("config_tests.yaml", "r") as stream:
    config_tests = yaml.safe_load(stream)

In [4]:
N_EPOCHS = 25
N_TRIALS = 50
BATCH_SIZE = 256

CONFIG = config_tests["assessment_proba_best"]

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [5]:
x1, y1 = utils.load_cifar_data("../data/cifar-10-batches-py/")

In [6]:
x1 /= 255

x, _, y, _ = train_test_split(x1, y1, train_size=0.2, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.1, random_state=42)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]

In [7]:
assessment_model = ClassifierModel(output_shape=n_classes, **models_hyperparameters["assessment_model"])

assessment_model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

assessment_model.fit(x_train, y_train, epochs=25, batch_size=BATCH_SIZE)

Epoch 1/25


2024-03-23 20:12:22.344252: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2024-03-23 20:12:22.344273: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2024-03-23 20:12:22.344278: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2024-03-23 20:12:22.344296: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-23 20:12:22.344305: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-03-23 20:12:23.203404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - accuracy: 0.1340 - loss: 2.4475
Epoch 2/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.2315 - loss: 2.1376
Epoch 3/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.2787 - loss: 2.0035
Epoch 4/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3145 - loss: 1.9248
Epoch 5/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3368 - loss: 1.8393
Epoch 6/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3383 - loss: 1.7968
Epoch 7/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3688 - loss: 1.7237
Epoch 8/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.3910 - loss: 1.6771
Epoch 9/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.4135 - loss: 1.6323
Epoch 10/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.4327 - loss: 1.5801
Epoch 11/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.4342 - loss: 1.5657
Epoch 12/25
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.4560 - l

In [8]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

samples_proba = utils.calculate_proba(
    assessment_model, x_train_sorted, y_train_sorted, counts, CONFIG["negative_loss"]
)

In [ ]:
results = {}
# test_models = ["test_model_1", "test_model_2", "test_model_3"]
test_models = ["test_model_1"]


for test_model in test_models:
    print(test_model)
    model_scores = []

    for _ in tqdm(range(N_TRIALS)):
        model = ClassifierModel(output_shape=n_classes, **models_hyperparameters[test_model])
        model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

        for i in range(N_EPOCHS):
            n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)

            samples_ids = utils.chose_samples(n_samples, samples_proba, CONFIG["order_type"])

            model.fit(
                x_train_sorted[samples_ids],
                y_train_sorted[samples_ids],
                # validation_data=(x_val, y_val),
                epochs=1,
                batch_size=BATCH_SIZE,
                verbose=0,
            )
            
            if CONFIG["progressive"]:
                samples_proba = utils.calculate_proba(
                    model, x_train_sorted, y_train_sorted, counts, CONFIG["negative_loss"]
                )

        _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=1)

        model_scores.append(accuracy)

    results[test_model] = model_scores

test_model_1


  0%|          | 0/50 [00:00<?, ?it/s]

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.5546 - loss: 1.3446


  2%|▏         | 1/50 [00:49<40:12, 49.24s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.5309 - loss: 1.3464


  4%|▍         | 2/50 [01:42<41:24, 51.75s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.5325 - loss: 1.4085


  6%|▌         | 3/50 [02:41<42:57, 54.84s/it]

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.5360 - loss: 1.4326


  8%|▊         | 4/50 [03:45<45:00, 58.70s/it]

In [ ]:
results

In [ ]:
sum(results["test_model_1"]) / N_TRIALS